# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
TODO

### Baseline Model
TODO

### Final Model
TODO

### Fairness Evaluation
To evaluate the fairness of my data model we decided to do a permutation test to find accuracy parity utilizing the difference in score between two randomly selected subsets of the data as our test statistic. Our null hypothesis was that the model was not biased in terms of predicting outage severity. Our alternative hypothesis was that the model was not fair. We decided to shuffle the population column and split up our data based on values above and below the median. We chose to utilize the population column to split up our data because it had no null values. Once we got the subsets of the data we ran our final model on each subset of the data and found the score of the model. Then we returned this absolute difference of score as the test statistic. We ran this proccess of getting the test statistic 1000 times to get a distribution of test statistics. We found our p-value based on the proportion of test statistics above a pre-determined threshold. We chose our threshold to be .02 so their was some room for error on the score since our final model was not perfect with a score of around .94. In the end we got a p-value of .02 so we were able to determine that our model was fair at signifigance level of .05.

# Code

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Binarizer

# Get Data

In [4]:
df = pd.read_csv('outage.csv')
df.columns = df.loc[4]
units = df.loc[5][1:]
df = df.drop([0,1,2,3,4,5]).drop(columns = ['variables']).reset_index(drop = True)
#df.isnull().sum(axis = 0).describe()
df['OUTAGE.START'] = df['OUTAGE.START.DATE']+" "+df['OUTAGE.START.TIME']
df['OUTAGE.RESTORATION'] = df['OUTAGE.RESTORATION.DATE']+" "+df['OUTAGE.RESTORATION.TIME']

#Converting to datetime object
df['OUTAGE.START'] = pd.to_datetime(df['OUTAGE.START'])
df['OUTAGE.RESTORATION'] = pd.to_datetime(df['OUTAGE.RESTORATION'])
df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.START.TIME'] = pd.to_datetime(df['OUTAGE.START.TIME'])
df['OUTAGE.RESTORATION.TIME'] = pd.to_datetime(df['OUTAGE.RESTORATION.TIME'])
#We can now drop the orginial columns 


df = df.set_index('OBS')
def convert(x):
    if x != x:
        return x
    try:
        out = float(x)
    except:
        out = x
    return out
#df = df.drop(columns = ['DEMAND.LOSS.MW','HURRICANE.NAMES','CAUSE.CATEGORY.DETAIL','CUSTOMERS.AFFECTED'])        
df = df.applymap(convert)
df = df[df['OUTAGE.START'].isnull() == False]
df = df[df['CLIMATE.REGION'].isnull() == False]
for col in df.columns:
    print(col + ' ' + str(type(df[col][0])))
df.head()

YEAR <class 'numpy.float64'>
MONTH <class 'numpy.float64'>
U.S._STATE <class 'str'>
POSTAL.CODE <class 'str'>
NERC.REGION <class 'str'>
CLIMATE.REGION <class 'str'>
ANOMALY.LEVEL <class 'numpy.float64'>
CLIMATE.CATEGORY <class 'str'>
OUTAGE.START.DATE <class 'pandas._libs.tslibs.timestamps.Timestamp'>
OUTAGE.START.TIME <class 'pandas._libs.tslibs.timestamps.Timestamp'>
OUTAGE.RESTORATION.DATE <class 'pandas._libs.tslibs.timestamps.Timestamp'>
OUTAGE.RESTORATION.TIME <class 'pandas._libs.tslibs.timestamps.Timestamp'>
CAUSE.CATEGORY <class 'str'>
CAUSE.CATEGORY.DETAIL <class 'float'>
HURRICANE.NAMES <class 'float'>
OUTAGE.DURATION <class 'numpy.float64'>
DEMAND.LOSS.MW <class 'numpy.float64'>
CUSTOMERS.AFFECTED <class 'numpy.float64'>
RES.PRICE <class 'numpy.float64'>
COM.PRICE <class 'numpy.float64'>
IND.PRICE <class 'numpy.float64'>
TOTAL.PRICE <class 'numpy.float64'>
RES.SALES <class 'numpy.float64'>
COM.SALES <class 'numpy.float64'>
IND.SALES <class 'numpy.float64'>
TOTAL.SALES <clas

4,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,2020-12-12 17:00:00,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,2020-12-12 18:38:00,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,2020-12-12 20:00:00,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,2020-12-12 04:30:00,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,2020-12-12 02:00:00,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00


### Baseline Model

In [5]:
def base_model(data):
    df = data.copy()
    df.fillna(df.mean(), inplace=True)
    X = df.drop(columns = ['OUTAGE.DURATION'])
    y = df['OUTAGE.DURATION']
    def fix_year(years):
        return years.apply(lambda x : x - 2000)
    year = ('year',FunctionTransformer(fix_year),['YEAR'])
    steps = [year]
    for col in data.columns:
        typ = type(df[col][0])
        if typ == str:
            steps.append((col,OneHotEncoder(),[col]))
    col_trans = ('col_trans',ColumnTransformer(steps))
    pl = Pipeline([col_trans,('lr', LinearRegression())])
    pl.fit(X,y)
    preds = pl.predict(X)
    return pl, preds, y

In [9]:
pl, preds, y = base_model(df)

rmse(preds,y)

5182.831773522928

In [10]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [11]:
temp = pd.DataFrame()
for col in df.columns:
    typ = type(df[col][0])
    if typ == str:
        temp[col] = df[col]
temp

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY
OBS,,,,,,
1,Minnesota,MN,MRO,East North Central,normal,severe weather
2,Minnesota,MN,MRO,East North Central,normal,intentional attack
3,Minnesota,MN,MRO,East North Central,cold,severe weather
4,Minnesota,MN,MRO,East North Central,normal,severe weather
5,Minnesota,MN,MRO,East North Central,warm,severe weather
...,...,...,...,...,...,...
1528,Idaho,ID,WECC,Northwest,warm,intentional attack
1529,Idaho,ID,WECC,Northwest,normal,system operability disruption
1530,North Dakota,ND,MRO,West North Central,cold,public appeal


In [12]:
temp = df.isnull().all(axis = 1).tolist()
temp

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [13]:
df.fillna(df.mean(), inplace=True)


In [14]:
df.isnull().sum()

4
YEAR                          0
MONTH                         0
U.S._STATE                    0
POSTAL.CODE                   0
NERC.REGION                   0
CLIMATE.REGION                0
ANOMALY.LEVEL                 0
CLIMATE.CATEGORY              0
OUTAGE.START.DATE             0
OUTAGE.START.TIME             0
OUTAGE.RESTORATION.DATE      49
OUTAGE.RESTORATION.TIME      49
CAUSE.CATEGORY                0
CAUSE.CATEGORY.DETAIL       468
HURRICANE.NAMES            1448
OUTAGE.DURATION               0
DEMAND.LOSS.MW                0
CUSTOMERS.AFFECTED            0
RES.PRICE                     0
COM.PRICE                     0
IND.PRICE                     0
TOTAL.PRICE                   0
RES.SALES                     0
COM.SALES                     0
IND.SALES                     0
TOTAL.SALES                   0
RES.PERCEN                    0
COM.PERCEN                    0
IND.PERCEN                    0
RES.CUSTOMERS                 0
COM.CUSTOMERS                 0
IND.CU

In [15]:
df[df['CLIMATE.REGION'].isnull()]

4,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,


In [16]:
def run_model(data):
    #convert items to float
    d = data.copy()
    def convert(x):
        if x != x:
            return x
        try:
            out = float(x)
        except:
            out = x
        return out
    d = d.applymap(convert)
    steps = []
    X = d.drop('Outcome', axis=1)
    y = d['Outcome']
    #the code below one hot encodes any string column
    for col in d.columns:
        typ = type(d[col][0])
        if typ == str:
            steps.append((col,OneHotEncoder(handle_unknown='ignore'),[col]))

    col_t = ('col_trans',ColumnTransformer(steps))
    pl = Pipeline([col_t,('tree', DecisionTreeClassifier())])

    fitted = pl.fit(X, y)
    accuracy = pl.score(X, y)
    return pl, accuracy

In [33]:
df = pd.read_csv('outage.csv')
df.columns = df.loc[4]
units = df.loc[5][1:]
df = df.drop([0,1,2,3,4,5]).drop(columns = ['variables']).reset_index(drop = True)
#df.isnull().sum(axis = 0).describe()
df['OUTAGE.START'] = df['OUTAGE.START.DATE']+" "+df['OUTAGE.START.TIME']
df['OUTAGE.RESTORATION'] = df['OUTAGE.RESTORATION.DATE']+" "+df['OUTAGE.RESTORATION.TIME']

#Converting to datetime object
df['OUTAGE.START'] = pd.to_datetime(df['OUTAGE.START'])
df['OUTAGE.RESTORATION'] = pd.to_datetime(df['OUTAGE.RESTORATION'])
df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.START.TIME'] = pd.to_datetime(df['OUTAGE.START.TIME'])
df['OUTAGE.RESTORATION.TIME'] = pd.to_datetime(df['OUTAGE.RESTORATION.TIME'])
df = df.set_index('OBS')
def convert(x):
    if x != x:
        return x
    try:
        out = float(x)
    except:
        out = x
    return out
#df = df.drop(columns = ['DEMAND.LOSS.MW','HURRICANE.NAMES','CAUSE.CATEGORY.DETAIL','CUSTOMERS.AFFECTED'])        
df = df.applymap(convert)
df = df[df['OUTAGE.START'].isnull() == False]
df = df[df['CLIMATE.REGION'].isnull() == False]
#we fill with 0 because these are usually intentional attacks
df[['OUTAGE.DURATION']] = df[['OUTAGE.DURATION']].fillna(value = 0)

#create a binarizer
transformer = Binarizer(threshold = 2550).fit(df[['OUTAGE.DURATION']])
#create outcomes based on data
outcomes = transformer.transform(df[['OUTAGE.DURATION']])
#define the outcomes
df['Outcome'] = outcomes
display(df[['OUTAGE.DURATION','Outcome']].head(5))
#Create a replacement dictionary 
season_dict = {1:'Winter', 2:'Winter',3:'Spring',4:'Spring',5:'Spring',
              6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',
              11:'Autumn',12:'Winter'}

#drop irrelevant columns 
test = df.drop(columns = ['YEAR','POSTAL.CODE','OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME',
                  'OUTAGE.RESTORATION.DATE','CAUSE.CATEGORY.DETAIL','HURRICANE.NAMES','POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL','OUTAGE.START','OUTAGE.RESTORATION', 'ANOMALY.LEVEL',
                         'RES.PRICE','COM.CUSTOMERS','COM.CUST.PCT','PC.REALGSP.USA',
                         'UTIL.REALGSP','AREAPCT_UC','IND.PRICE','PCT_WATER_TOT','CLIMATE.CATEGORY',
                         'PC.REALGSP.STATE','AREAPCT_URBAN','COM.PRICE','PI.UTIL.OFUSA',
                          'PCT_LAND','PCT_WATER_INLAND','RES.PERCEN','COM.PERCEN','UTIL.CONTRI'])

# replace the months with season and add a column to the dataset
test['SEASON'] = test['MONTH'].replace(season_dict)
# run the model 
pl, acc = run_model(test)
acc
west = ['WA','OR','CA','ID','NV','MT','WY','UT','AZ','CO','NM']
midwest = ['ND','MN','WI','MI','SD','NE','KS','IA','MO','IL','IN','OH']
south = ['TX','OK','AR','LA','MS','TN','KY','AL','GA','FL','SC','NC','VA','WV','MD','DE']
northeast = ['PA','NJ','NY','CT','MA','RI','VT','NH','ME']
test_df = df.drop(columns = ['YEAR','POSTAL.CODE','OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME',
                  'OUTAGE.RESTORATION.DATE','CAUSE.CATEGORY.DETAIL','HURRICANE.NAMES','POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL','OUTAGE.START','OUTAGE.RESTORATION', 'ANOMALY.LEVEL',
                         'RES.PRICE','COM.CUSTOMERS','COM.CUST.PCT','PC.REALGSP.USA',
                         'UTIL.REALGSP','AREAPCT_UC','IND.PRICE','PCT_WATER_TOT','CLIMATE.CATEGORY',
                         'PC.REALGSP.STATE','AREAPCT_URBAN','COM.PRICE','PI.UTIL.OFUSA',
                          'PCT_LAND','PCT_WATER_INLAND','RES.PERCEN','COM.PERCEN','UTIL.CONTRI'])

west_dict = {i:"west" for i in west}
midwest_dict = {i:"midwest" for i in midwest}
south_dict = {i:"south" for i in south}
northeast_dict = {i:"northeast" for i in northeast}
d = {**west_dict, **midwest_dict, **south_dict, **northeast_dict}
test_df['REGION'] = df['POSTAL.CODE'].map(d)
test_df['REGION']=test_df['REGION'].fillna(value = 'null')
p, a = run_model(test_df)
a

4,OUTAGE.DURATION,Outcome
OBS,,
1,3060.0,1.0
2,1.0,0.0
3,3000.0,1.0
4,2550.0,0.0
5,1740.0,0.0


0.8289473684210527

In [18]:
dec_tree = pl['tree']
#We can use the dictionary to see what cols are necessary! 
dict(zip(X.columns, dec_tree.feature_importances_))

NameError: name 'X' is not defined

### Final Model

In [20]:
def run_final_model(data):
    #convert items to float
    d = data.copy()
    def convert(x):
        if x != x:
            return x
        try:
            out = float(x)
        except:
            out = x
        return out
    d = d.applymap(convert)
    steps = [('log-scale',FunctionTransformer(np.log),['TOTAL.REALGSP'])]
    X = d.drop('Outcome', axis=1)
    y = d['Outcome']
    #the code below one hot encodes any string column
    for col in d.columns:
        typ = type(d[col][0])
        if typ == str:
            steps.append((col,OneHotEncoder(handle_unknown='ignore'),[col]))

    col_t = ('col_trans',ColumnTransformer(steps))
    pl = Pipeline([col_t,('tree', DecisionTreeClassifier(max_depth = 25))])

    fitted = pl.fit(X, y)
    accuracy = pl.score(X, y)
    return pl, accuracy
final_pl, acc = run_final_model(test)
acc

0.9368421052631579

### Fairness Evaluation

In [81]:
def get_t_stat(data):
    '''Function to get a t_stat for difference of scores between 2 random subsets of the data'''
    #Create copy of the data so original df is not manipulated
    df = data.copy()
    #Shuffles the column 'POPULATION'
    df['POPULATION'] = np.random.permutation(df['POPULATION'].values)
    #Split into 2 subsets of the data to do permutation test based on whether or not the values of 'POPULATION'
    #based on whether they are above or below median.
    temp = data.copy()
    median = temp['POPULATION'].sort_values()[test.shape[0] // 2]
    df1 = df[df['POPULATION'] >= median]
    df2 = df[df['POPULATION'] < median]
    #compare the absolute difference of score for each subset as my t_stat
    pl1, score1 = run_final_model(df1)
    pl2, score2 = run_final_model(df2)
    t_stat = abs(score1 - score2)
    return t_stat
t_stats = []
#Create a distribution of N t_stats
N = 1000
for i in range(N):
    t_stats.append(get_t_stat(test))
#Get p_value based on amount of t_stats over a predetermined threshold
threshold = .02
p_value = np.count_nonzero(np.array(t_stats) > threshold) / test.shape[0]
p_value

0.02631578947368421

In [79]:
threshold = .02
p_value = np.count_nonzero(np.array(t_stats) > threshold) / test.shape[0]
p_value

0.02631578947368421

In [44]:
temp['CUSTOMERS.AFFECTED'] == test['CUSTOMERS.AFFECTED']


OBS
1       False
2       False
3       False
4       False
5       False
        ...  
1528     True
1529    False
1530    False
1532    False
1533    False
Name: CUSTOMERS.AFFECTED, Length: 1520, dtype: bool

In [51]:
np.nan > 1

False

In [55]:
test.isnull().sum(axis =0 )

4
MONTH                   0
U.S._STATE              0
NERC.REGION             0
CLIMATE.REGION          0
CAUSE.CATEGORY          0
OUTAGE.DURATION         0
DEMAND.LOSS.MW        702
CUSTOMERS.AFFECTED    441
TOTAL.PRICE            13
RES.SALES              13
COM.SALES              13
IND.SALES              13
TOTAL.SALES            13
IND.PERCEN             13
RES.CUSTOMERS           0
IND.CUSTOMERS           0
TOTAL.CUSTOMERS         0
RES.CUST.PCT            0
IND.CUST.PCT            0
PC.REALGSP.REL          0
PC.REALGSP.CHANGE       0
TOTAL.REALGSP           0
POPULATION              0
Outcome                 0
SEASON                  0
dtype: int64

In [56]:
test['POPULATION'].sort_values()

OBS
1       5348119.0
2       5457125.0
3       5310903.0
4       5380443.0
5       5489594.0
          ...    
1528    1680026.0
1529    1680026.0
1530     685326.0
1532     807067.0
1533     807067.0
Name: POPULATION, Length: 1520, dtype: float64